In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox import options
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd


In [25]:
options = Options()
options.headless = False #enxergar o que ele tá fazendo

In [26]:
# Carrega o webdriver para firefox
navegador = webdriver.Firefox(options=options)

In [28]:
#Define o site que vai entrar
link = "https://app.uff.br/iduff/"

In [29]:
#efetuando o login
def login():
    navegador.get(url=link)
    WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.ID, "login:id")))
    inputUsuario = navegador.find_element(by=By.ID,value="login:id")
    inputUsuario.send_keys("02485381100")
    inputUsuario = navegador.find_element(by=By.ID,value="login:senha")
    inputUsuario.send_keys("m1e2ll3o4")
    #entrando no login
    buttonLogin = navegador.find_element(by=By.ID, value="login:btnLogar")
    buttonLogin.click()
    WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.LINK_TEXT, "Funcionário - 1253936")))
    linkFuncionario = navegador.find_element(by=By.LINK_TEXT, value="Funcionário - 1253936")
    linkFuncionario.click()
    WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.LINK_TEXT, "Gerenciar Alunos")))
    linkAlunos = navegador.find_element(by=By.LINK_TEXT, value="Gerenciar Alunos")
    linkAlunos.click()

In [30]:
#leitura da tabela de alunos, cujos dados serão extraídos
def carrega_dados():
    dados_alunos = pd.read_excel('C:/Users/salut/Desktop\MBA_CD/artigo_final/lista_todos_alunos.xlsx')
    # Filtrar as linhas que começam com os prefixos desejados
    prefixos_desejados = ['113', '114', '115', '213', '214', '215']
    situacao = dados_alunos["Situação"].isin(["Formado", "Cancelamento por Abandono", "Cancelamento por Insuficiência de Aproveitamento"])
    df = dados_alunos[dados_alunos["Matrícula"].str.startswith(tuple(prefixos_desejados)) & situacao]
    return df

In [31]:
login()
dados_alunos_filtrado = carrega_dados()

In [32]:
i=1
# Iterar sobre cada linha do DataFrame
for index, row in dados_alunos_filtrado.iterrows():
    print ("Linha " + str(i) + " de " + str(len(dados_alunos_filtrado)))
    i = i+1

    WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.ID, "gerenciarAlunos:cboBusca")))
    
    #na aba gerenciar aluno, mudar para a opção Nome
    tipobuscaAluno = navegador.find_element(by=By.ID, value="gerenciarAlunos:cboBusca")
    tipobuscaAluno.click()
    tipobuscaAluno.send_keys("Nome")

    # Nome do aluno na linha atual
    aluno = row['Nome']

    # Inserir o nome do aluno no campo de busca
    busca_aluno = navegador.find_element(by=By.ID, value="gerenciarAlunos:busca")

    # Esperar até que o campo de busca esteja visível
    WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.ID, "gerenciarAlunos:busca")))

    # Inserir o nome do aluno no campo de busca
    busca_aluno.send_keys(aluno)

    # Clicar no botão de busca
    busca_aluno_button = navegador.find_element(by=By.ID, value="gerenciarAlunos:btnBuscar")
    busca_aluno_button.click()

    # Esperar até que o botão "Ver" esteja visível
    WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.ID, "gerenciarAlunos3:tabelaAlunos:0:btnVerAcompanhamento")))

    # Clicar em "Ver"
    ver_acompanhamento = navegador.find_element(by=By.ID, value="gerenciarAlunos3:tabelaAlunos:0:btnVerAcompanhamento")
    ver_acompanhamento.click()

    WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.ID, "j_id147_lbl"))) 

    # Seguir para dados pessoais
    ver_pessoais = navegador.find_element(by=By.ID, value="j_id147_lbl")
    ver_pessoais.click()

    WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.ID, "decCor:identificacaoCor"))) 

    # Coletar as informações
    cor = navegador.find_element(by=By.ID, value="decCor:identificacaoCor").text
    municipio_residencia = navegador.find_element(by=By.ID, value="decCidade:identificacaoEnderecoCidade").text
    nascimento = navegador.find_element(by=By.ID, value="decDataNascimento:identificacaoDataNascimento").text

    # Atualizar o DataFrame
    row['Cor'] = cor
    row['Municipio_Residencia'] = municipio_residencia
    row['Nascimento'] = nascimento


    # Voltando para a página de busca
    voltar_para_busca = navegador.find_element(By.LINK_TEXT, "Buscar outro Aluno")
    navegador.execute_script("arguments[0].click();", voltar_para_busca)

        


Linha 1 de 121
Linha 2 de 121
Linha 3 de 121
Linha 4 de 121
Linha 5 de 121
Linha 6 de 121


KeyboardInterrupt: 

In [ ]:

dados_alunos_filtrado.to_excel("PrimeiraExtracao.xlsx")
